<h2>Churn de Clientes - Bank Dataset</h2>

In [ ]:
## ----------------------------------------------------------------
#* Link do dataset
    #  https://www.kaggle.com/datasets/mathchi/churn-for-bank-customers

#* Descrição das variáveis
    #* RowNumber
    #Número da linha do dataset
    #* CustomerId
    #Número randômico e único que identifica o cliente
    #* Surname
    #Sobrenome do cliente
    #* CreditScore
    #Pontuação de crédito atribuida ao cliente
    #* Geography
    #Localização do cliente
    #* Gender
    #Gênero do cliente
    #* Age
    #Idade do cliente
    #* Tenure
    #Fidelidade do cliente | Quantidade de anos que o cliente está na instituição
    #* Balance
    #Saldo do cliente
    #* NumOfProducts
    #Quantidade de produtos que o cliente comprou no banco
    #* HasCrCard
    #Se o cliente possui ou não cartão de crédito sendo 1|Sim e 0|Não
    #* IsActiveMember
    #  - Se o cliente é ativo ou não no banco
    #* EstimatedSalary
    #  - Renda do cliente
    #*Exited
    #  - Se o cliente deixou ou não a instituição sendo 1|Sim e 0|Não  
## ----------------------------------------------------------------

<h5>1. Importação dos pacotes </h5>

In [ ]:
## ----------------------------------------------------------------
#* Manipulação de dados
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats as st
#* DataViz
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
import ydata_profiling
#! Utilizar ydata_profiling em Jupyter notebook
# import sys
# !{sys.executable} -m pip install -U ydata-profiling[notebook]
# !jupyter nbextension enable --py widgetsnbextension
#
from ydata_profiling import ProfileReport
#! Pacotes não utilizados no momento
#import plotly.express as px
#import plotly.io as pio
#import chart_studio.plotly as py
#import plotly.graph_objects as go
#from plotly.subplots import make_subplots
#* Machine Learning
## ----------------------------------------------------------------

<h5>2. Leitura e tratamento do dataset </h5>

In [ ]:
df_original = pd.read_csv(filepath_or_buffer = 'churn.csv', header = 0)
df_original.head()

In [ ]:
#Excluir colunas que não serão utilizadas
trash_colunas = {'RowNumber', 'CustomerId', 'Surname'}
#
df = df_original.drop(columns = trash_colunas)
del(trash_colunas)
del(df_original)
df.head()

In [ ]:
#Criar lista com nome das novas variáveis
novas_colunas = {
    'CreditScore': 'Score',
    'Geography': 'Pais',
    'Gender': 'Genero',
    'Age': 'Idade',
    'Tenure': 'Fidelidade',
    'Balance': 'Saldo',
    'NumOfProducts': 'QtdProdutos',
    'HasCrCard': 'TemCartao',
    'IsActiveMember': 'ClienteAtivo',
    'EstimatedSalary': 'Renda',
    'Exited': 'Churn'
    }
#renomeando as variáveis
df.rename(columns = novas_colunas, inplace = True)
#
del(novas_colunas)
#
df.head()

<h4>3. Análise Exploratória </h4>

In [ ]:
# #Criação de perfil para geração de relatório Profiling
# profile = ProfileReport(df, title = 'Bank Churn')
# #Funções para utilização do pacote ydata_profiling em Jupyter
# profile.to_widgets()
# profile.to_notebook_iframe()

In [ ]:
# #Salvando o relatório em html
# profile.to_file('churn_Profile.html')

In [ ]:
#Informações sobre missing values e tipo das variáveis
df.info()

In [ ]:
#Criar lista com o nome das colunas (variáveis)
variaveis = df.columns.to_list()
variaveis

In [ ]:
#Mostrar a quantidade de registros em cada variável
df[variaveis].nunique()

In [ ]:
# Listar variáveis quantitativa contínuas
var_quantitativas_continuas = [
    variavel for variavel in df.columns
        if df[variavel].dtype == 'float64' and variavel not in 'Churn']
var_quantitativas_continuas

In [ ]:
# Mostrar informaçõs estatísticas das variáveis quantitativas contínuas
df[var_quantitativas_continuas].describe().T

In [ ]:
# Listar variáveis quantitativa discretas
var_quantitativas_discretas = [
    variavel for variavel in df.columns
        if df[variavel].dtype == 'int64' and df[variavel].nunique() > 2 and variavel not in 'Churn']
var_quantitativas_discretas

In [ ]:
# Mostrar informaçõs estatísticas das variáveis quantitativas discretas
df[var_quantitativas_discretas].describe().T

In [ ]:
# Listar variáveis qualitativas nominais
var_qualitativas_nominais = [
    variavel for variavel in df.columns
        if df[variavel].dtype == 'object' and variavel not in 'Churn']
var_qualitativas_nominais

In [ ]:
# Mostrar informaçõs estatísticas das variáveis qualitativas nominais
df[var_qualitativas_nominais].describe().T

In [ ]:
#Alterando o conteúdo das variáveis categóricas
#* Localização
df['Pais'].value_counts()

In [ ]:
#Alterando os valores da variável #Localização
df['Pais'].replace({
    'France' : 'França',
    'Germany' : 'Alemanha',
    'Spain' : 'Espanha'
}, inplace = True)
#
df['Pais'].unique()

In [ ]:
#Alterando o conteúdo das variáveis categóricas
#* Gênero
df['Genero'].value_counts()

In [ ]:
#Alterando os valores da variável #Localização
#* Gênero
df['Genero'].replace({
    'Male' : 'Masculino',
    'Female' : 'Feminino'
}, inplace = True)
#
df['Genero'].unique()

In [ ]:
# Listar variáveis qualitativas ordinais
var_qualitativas_ordinais = [
    variavel for variavel in df.columns
        if df[variavel].dtype == 'int64' and df[variavel].nunique() == 2 and variavel not in 'Churn']
var_qualitativas_ordinais

<h5>3.1. Análise univariada</h5>

In [ ]:
#IQR - https://www.statisticshowto.com/probability-and-statistics/interquartile-range/
#Lower and Upper Fence - https://www.statisticshowto.com/upper-and-lower-fences/

In [ ]:
# Definição do Tema do Seaborn
sns.set_theme(
    context = 'paper', style = 'whitegrid', palette = 'muted',
    font = 'sans-serif', font_scale = 1, color_codes = True, 
    rc = None
)

3.1.1. Score de Crédito

In [ ]:
#Criação dos subplots
fig, axs = plt.subplots(ncols = 1, nrows = 2, sharex=True, height_ratios=[0.8, 0.2])
#Título da imagem
plt.suptitle('Distribuição do Score de Crédito')
#Gráfico 1 - Histograma
sns.histplot(x = df['Score'], kde = True, ax = axs[0], stat = 'count', cumulative = False)
#Gráfico 2 - Boxplot
sns.boxplot(x = df['Score'], fliersize = 2, notch = True, ax = axs[1])

3.1.2. Idade

In [ ]:
#Criação dos subplots
fig, axs = plt.subplots(ncols = 1, nrows = 2, sharex=True, height_ratios=[0.8, 0.2])
#Título da imagem
plt.suptitle('Distribuição por Idade')
#Gráfico 1 - Histograma
sns.histplot(x = df['Idade'], kde = True, ax = axs[0], stat = 'count', cumulative = False)
#Gráfico 2 - Boxplot
sns.boxplot(x = df['Idade'], fliersize = 2, notch = True, ax = axs[1])

3.1.3. Tempo de fidelidade

In [ ]:
#Criação dos subplots
fig, axs = plt.subplots(ncols = 1, nrows = 2, sharex=True, height_ratios=[0.8, 0.2])
#Título da imagem
plt.suptitle('Distribuição por tempo de fidelidade do cliente')
#Gráfico 1 - Histograma
sns.histplot(x = df['Fidelidade'], kde = False, ax = axs[0], stat = 'count', cumulative = False)
#Gráfico 2 - Boxplot
sns.boxplot(x = df['Fidelidade'], fliersize = 2, notch = True, ax = axs[1])

3.1.4. Saldo

In [ ]:
#Criação dos subplots
fig, axs = plt.subplots(ncols = 1, nrows = 2, sharex=True, height_ratios=[0.8, 0.2])
#Título da imagem
plt.suptitle('Distribuição por saldo em conta')
#Gráfico 1 - Histograma
sns.histplot(x = df['Saldo'], kde = False, ax = axs[0], stat = 'count', cumulative = False)
#Gráfico 2 - Boxplot
sns.boxplot(x = df['Saldo'], fliersize = 2, notch = True, ax = axs[1])

3.1.5. Quantidade de produtos

In [ ]:
#Criação dos subplots
fig, axs = plt.subplots(ncols = 1, nrows = 2, sharex=True, height_ratios=[0.8, 0.2])
#Título da imagem
plt.suptitle('Distribuição por quantidade de produtos')
#Gráfico 1 - Histograma
sns.histplot(x = df['QtdProdutos'], kde = False, ax = axs[0], stat = 'count', cumulative = False)
#Gráfico 2 - Boxplot
sns.boxplot(x = df['QtdProdutos'], fliersize = 2, notch = True, ax = axs[1])

3.1.6. Salário Estimado

In [ ]:
#Criação dos subplots
fig, axs = plt.subplots(ncols = 1, nrows = 2, sharex=True, height_ratios=[0.8, 0.2])
#Título da imagem
plt.suptitle('Faixa salarial')
#Gráfico 1 - Histograma
sns.histplot(x = df['Renda'], kde = False, ax = axs[0], stat = 'count', cumulative = False)
#Gráfico 2 - Boxplot
sns.boxplot(x = df['Renda'], fliersize = 2, notch = True, ax = axs[1])

3.1.7. Cartão de Crédito

In [ ]:
#Título da imagem
plt.suptitle('Cartão de Crédito')
ax = sns.countplot(data = df, x = df['TemCartao'])
#Inclusão dos rótulos
ax.bar_label(ax.containers[0])

3.1.8. Clientes ativos e inativos

In [ ]:
#Título da imagem
plt.suptitle('Clientes ativos e inativos')
ax = sns.countplot(data = df, x = df['ClienteAtivo'])
#Inclusão dos rótulos
ax.bar_label(ax.containers[0])

3.1.9. Churn

In [ ]:
#Título da imagem
plt.suptitle('Churn')
ax = sns.countplot(data = df, x = df['Churn'])
#Inclusão dos rótulos
ax.bar_label(ax.containers[0])

3.1.10. País de residência

In [ ]:
#Título da imagem
plt.suptitle('Clientes por Páis')
ax = sns.countplot(data = df, x = df['Pais'])
#Inclusão dos rótulos
ax.bar_label(ax.containers[0])

3.1.11. Gênero

In [ ]:
#Título da imagem
plt.suptitle('Clientes por gênero')
ax = sns.countplot(data = df, x = df['Genero'])
#Inclusão dos rótulos
ax.bar_label(ax.containers[0])

<h5>3.2. Análise bivariada</h5>

Matriz de Correlação

In [ ]:
#Matriz de correlação
corr = df.corr()
corr

In [ ]:
#Criação da máscara a ser utilizada no heatmap de correlação
mask = np.triu(np.ones_like(corr, dtype = bool))
#Configuração da paleta de divergência
paleta_divergencia = sns.diverging_palette(h_neg = 250, h_pos = 20, s = 75, l = 50, sep = 1, n = 6, center = 'light', as_cmap = False)
paleta_divergencia

In [ ]:
#Criação do gráfico da matriz de correlação
f, ax = plt.subplots(figsize = (9, 7))
sns.heatmap(corr, mask = mask, cmap = paleta_divergencia, linewidths= 0.4, cbar=True, annot = True)
plt.title('Matriz de Correlação')

3.2.1. Churn x Idade

In [ ]:
#Título da imagem
plt.suptitle('Idade x Churn')
#Análise bivariada
sns.boxplot(df, x = df['Idade'], y = df['Churn'].replace({1 : 'Saiu - (1)', 0: 'NaoSaiu - (0)'}), fliersize = 2, notch = True)

3.2.1. Churn x Saldo

In [ ]:
#Título da imagem
plt.suptitle('Saldo x Churn')
#Análise bivariada
sns.boxplot(df, x = df['Saldo'], y = df['Churn'].replace({1 : 'Saiu - (1)', 0: 'NaoSaiu - (0)'}), fliersize = 2, notch = True)

In [ ]:
#Definição do estilo dos gráficos
# plt.style.use('seaborn-v0_8')
# pio.templates

In [ ]:
# Definição do template dos gráficos
# pio.templates.default = 'seaborn'

In [ ]:
# #Criando a figura para plotagem dos gráficos
# fig = make_subplots(rows = 2, shared_xaxes = True, vertical_spacing = 0.05, row_heights = [0.6, 0.4])

# #Gráfico 1 - Histograma
# fig.add_trace(go.Histogram(x = df.Score), row = 1, col = 1)

# #Gráfico 2 - Boxplot
# fig.add_trace(go.Box(x = df.Score, name = '', orientation = 'h'), row = 2, col=1)

# #Alterações de layout    
# fig.update_layout(title = 'Score de Crédito', showlegend = False, title_font_size = 18)

# #Configuração do eixo X    
# fig.update_xaxes(range = [300, 900])

# #Mostrar o gráfico
# fig.show()

In [ ]:
# df.Idade.describe()

In [ ]:
# #Criando a figura para plotagem dos gráficos
# fig = make_subplots(rows = 2, shared_xaxes = True, vertical_spacing = 0.05, row_heights = [0.6, 0.4])

# #Gráfico 1 - Histograma
# fig.add_trace(go.Histogram(x = df.Idade), row = 1, col = 1)

# #Gráfico 2 - Boxplot
# fig.add_trace(go.Box(x = df.Idade, name = '', orientation = 'h'), row = 2, col=1)

# #Alterações de layout    
# fig.update_layout(title = 'Idade', showlegend = False, title_font_size = 18)

# #Configuração do eixo X    
# fig.update_xaxes(range = [0, 100])

# #Mostrar o gráfico
# fig.show()

In [ ]:
# df.Fidelidade.describe()

In [ ]:
# #Criando a figura para plotagem dos gráficos
# fig = make_subplots(rows = 2, shared_xaxes = True, vertical_spacing = 0.05, row_heights = [0.6, 0.4])

# #Gráfico 1 - Histograma
# fig.add_trace(go.Histogram(x = df.Fidelidade), row = 1, col = 1)

# #Gráfico 2 - Boxplot
# fig.add_trace(go.Box(x = df.Fidelidade, name = '', orientation = 'h'), row = 2, col=1)

# #Alterações de layout    
# fig.update_layout(title = 'Tempo de fidelidade (em anos)', showlegend = False, title_font_size = 18)

# #Configuração do eixo X    
# fig.update_xaxes(range = [-5, 15])

# #Mostrar o gráfico
# fig.show()

In [ ]:
# df.Saldo.describe()

In [ ]:
# #Criando a figura para plotagem dos gráficos
# fig = make_subplots(rows = 2, shared_xaxes = True, vertical_spacing = 0.05, row_heights = [0.6, 0.4])

# #Gráfico 1 - Histograma
# fig.add_trace(go.Histogram(x = df.Saldo), row = 1, col = 1)

# #Gráfico 2 - Boxplot
# fig.add_trace(go.Box(x = df.Saldo, name = '', orientation = 'h'), row = 2, col=1)

# #Alterações de layout    
# fig.update_layout(title = 'Saldo', showlegend = False, title_font_size = 18)

# #Configuração do eixo X    
# fig.update_xaxes(range = [-10000, 300000])

# #Mostrar o gráfico
# fig.show()

In [ ]:
# df_numericas

In [ ]:
# df.QtdProdutos.describe()

In [ ]:
# #Criando a figura para plotagem dos gráficos
# fig = make_subplots(rows = 2, shared_xaxes = True, vertical_spacing = 0.05, row_heights = [0.6, 0.4])

# #Gráfico 1 - Histograma
# fig.add_trace(go.Histogram(x = df.QtdProdutos), row = 1, col = 1)

# #Gráfico 2 - Boxplot
# fig.add_trace(go.Box(x = df.QtdProdutos, name = '', orientation = 'h'), row = 2, col=1)

# #Alterações de layout    
# fig.update_layout(title = 'Quantidade de Produtos', showlegend = False, title_font_size = 18)

# #Configuração do eixo X    
# fig.update_xaxes(range = [0, 5])

# #Mostrar o gráfico
# fig.show()

In [ ]:
# #Definições dos quartis, mediana e limites e contabilização de outliers
# q1_Score = np.nanpercentile(df.Score, 25)
# q3_Score = np.nanpercentile(df.Score, 75)
# media_Score = np.mean(df.Score)
# iqr_Score = q3_Score - q1_Score
# lower_f_Score = q1_Score - (1.5 * iqr_Score)
# upper_f_Score = q3_Score + (1.5 * iqr_Score)

# #Quantidade de outliers
# Score_outlier = []
# for num in df.Score:
#     if num < lower_f_Score:
#         Score_outlier.append(num)
#     if num > upper_f_Score:
#         Score_outlier.append(num)

# #Quantidade de não outliers
# Score_non_outlier = []
# for num in df.Score:
#     if num >= lower_f_Score:
#         Score_non_outlier.append(num)
#     if num <= upper_f_Score:
#         Score_non_outlier.append(num)

In [ ]:
# print('---Score de Crédito---')
# print('Q1:', q1_Score)
# print('Q3:', q3_Score)
# print('Intervalo Interquartil (IQR):', iqr_Score)
# print('Média:', media_Score)
# print('Lower Fence:', lower_f_Score)
# print('Upper Fence:', upper_f_Score)
# print('Soma de valores únicos:', len(Score_outlier))
# print('Menor valor:', np.min(Score_non_outlier))
# print('Maior valor:', np.max(Score_non_outlier))
# print('Outliers:', len(Score_outlier))
# print('Não outliers:', len(Score_non_outlier))

In [ ]:
# #Criando figura para inserção de dois gráficos
# fig, ax = plt.subplots(2, 1, gridspec_kw={'height_ratios': [5,1]}, sharex=True)
# #Histograma
# sns.histplot(x = df.Score, kde=True, ax=ax[0])
# ax[0].axvline(x = q1_Score, linestyle='-', linewidth=1.5, color='black')
# ax[0].axvline(x = q3_Score, linestyle='-', linewidth=1.5, color='black')
# ax[0].axvline(x = np.nanpercentile(df.Score, 50), linestyle='-', linewidth=1.5, color='black')
# ax[0].axvline(x = np.mean(df.Score), linestyle=':', linewidth=2, color='red')
# #Boxplot
# sns.boxplot(x = df.Score, ax=ax[1])

In [ ]:
# def outlier(x):
#     q1 = np.nanpercentile(x, 25)
#     q3 = np.nanpercentile(x, 75)
#     iqr = q3 - q1
#     lower_fence = q1 - (1.5 * iqr)
#     upper_fence = q3 + (1.5 * iqr)
    
#     outlier = []
#     for num in x:
#         if num < lower_fence:
#             outlier.append(num)
#         if num > upper_fence:
#             outlier.append(num)
            
#     non_outlier = []
#     for num in x:
#         if num >= lower_fence:
#             non_outlier.append(num)
#         if num <= upper_fence:
#             non_outlier.append(num)
    
#     print(f'sum of unique value : {len(outlier)}' )
#     print(f'lower_value: {np.min(non_outlier)}')
#     print(f'upper_vaue: {np.max(non_outlier)}')

In [ ]:
# norm_data = []
# for y,x in enumerate(df_numericas,1):
#     print(f'{y}. {x}')
#     fig, ax = plt.subplots(2,1, sharex=True, gridspec_kw={'height_ratios':[5,1]})
#     sns.histplot(data=df, x=x, kde=True, ax=ax[0])
#     ax[0].axvline(np.nanpercentile(df[x], 25), c='red',linestyle='--')
#     ax[0].axvline(np.nanpercentile(df[x], 50), c='red',linestyle='--')
#     ax[0].axvline(np.nanpercentile(df[x], 75), c='red',linestyle='--')
#     ax[0].axvline(np.mean(df[x]), c='blue',linestyle='--')
    
#     sns.boxplot(data=df, x=x, ax=ax[1])
#     ax[1].set_xlabel(f'{x}')
#     plt.show()
#     print(' ')
#     df[x].describe()
#     print(' ')
#     outlier(df[x])
#     print(' ')
#     sk, pval = st.skewtest(df[x])
#     if pval < 0.05:
#         print(f'Data skewed with skew is {sk}')
#     else:
#         print(f'Data not skewed')
#     print(' ')
#     norm, pval = st.normaltest(df[x])
#     if pval < 0.05:
#         print(f'Data not normal')
#     else:
#         print(f'Data normal')
#         norm_data.append(x)
#     print('-'*50)
#     print(' ')